**softmax**

- $\sigma$	=	softmax
- $\vec{z}$	=	input vector
- $e^{z_{i}}$	=	standard exponential function for input vector
- $K$	=	number of classes in the multi-class classifier
- $e^{z_{j}}$	=	standard exponential function for output vector
- $e^{z_{j}}$	=	standard exponential function for output vector

------------
![image](/Users/dimtriospanagoulias/Downloads/NLP_UNIPI/nlp_lab/Neuron2transformer/img_blog_image1_inline_(2).webp)



Mathematical Proof:

- Let's call the max value c
    - The modified formula is: $\frac{e^{x_i-c}}{\sum_{j=1}^n e^{x_j-c}}$
    - This simplifies to: $\frac{e^{x_i}/e^c}{\sum_{j=1}^n e^{x_j}/e^c} = \frac{e^{x_i}}{\sum_{j=1}^n e^{x_j}}$

- - Why Use This Form:

    - Prevents numerical overflow
    - Avoids inf values when dealing with large numbers
    - More stable training in deep learning
